In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from bdtp4.parser import parse
import json
import re

dataset = {}
for e in filter(lambda x: x, parse('../data/amazon-meta.txt')):
    e = dict((k.decode() if type(k) is bytes else k, v.decode() if type(v) is bytes else v) for k, v in e.items())
    
    for k in ['id', 'salesrank']:
        if k in e:
            e[k] = int(e[k])
    
    if 'categories' in e:
        e['categories'] = re.findall(r'(?<=\|)([^\|\[]+)\[(\d+)\]', ''.join(e['categories']))

    if 'similar_items' in e:
        e['similar'] = e.pop('similar_items')
    
    dataset[e['id']] = e

In [8]:
import random
random.choice(list(dataset.values()))

{'id': 533871,
 'ASIN': '0788199706',
 'title': 'Think Like a Champion',
 'group': 'Book',
 'salesrank': 1612881,
 'similar': ['0834210215',
  '076192566X',
  '0809231719',
  '0060579447',
  '0425141756'],
 'categories': [('Books', '283155'),
  ('Specialty Stores', '504360'),
  ('jp-unknown1', '1061348'),
  ('Books', '283155'),
  ('Subjects', '1000'),
  ('Sports', '26'),
  ('General', '11086921')],
 'reviews': [{'time': 935294400.0,
   'customer_id': 'A2WKAL0VESHPYO',
   'rating': 5,
   'votes': 2,
   'helpful': 2},
  {'time': 935467200.0,
   'customer_id': 'A24BI5J96Q5DDD',
   'rating': 5,
   'votes': 1,
   'helpful': 1},
  {'time': 935899200.0,
   'customer_id': 'A7MM8RMX79EJD',
   'rating': 5,
   'votes': 2,
   'helpful': 2},
  {'time': 941601600.0,
   'customer_id': 'ANZZU2PGU9IC7',
   'rating': 5,
   'votes': 4,
   'helpful': 3},
  {'time': 947390400.0,
   'customer_id': 'A4QT91PMRSZWO',
   'rating': 5,
   'votes': 6,
   'helpful': 5},
  {'time': 951278400.0,
   'customer_id': 'A2

In [14]:
import json

with open('../data/dataset.json', 'w') as file:
    json.dump(dataset, file)